In [129]:
import folium
import pandas as pd

filePath1 = "../data/east_coast_vessels.csv"
vessel_df = pd.read_csv(filePath1)

filePath2 = "../data/hurdat.csv"
hurricane_df = pd.read_csv(filePath2)

In [130]:
vessel_df.columns

Index(['Unnamed: 0', 'MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG',
       'Heading', 'VesselName', 'IMO', 'CallSign', 'VesselType', 'Status',
       'Length', 'Width', 'Draft', 'Cargo', 'TransceiverClass'],
      dtype='object')

In [131]:
hurricane_df.shape

(346, 23)

In [132]:
def convert_coordinates(coord):
    # Remove leading/trailing whitespace
    coord = coord.strip()

    # Check if the coordinate has direction 'N', 'S', 'E', 'W' and convert accordingly
    if coord[-1] == 'N':
        return float(coord[:-1])  # North is positive
    elif coord[-1] == 'S':
        return -float(coord[:-1])  # South is negative
    elif coord[-1] == 'E':
        return float(coord[:-1])  # East is positive
    elif coord[-1] == 'W':
        return -float(coord[:-1])  # West is negative
    else:
        return float(coord)  # Already a float value


In [133]:
# Clean the Latitude and Longitude in the hurricane dataset
hurricane_df['Latitude'] = hurricane_df['Latitude'].apply(convert_coordinates)
hurricane_df['Longitude'] = hurricane_df['Longitude'].apply(convert_coordinates)

# Convert 'BaseDateTime' to datetime format in the vessel dataset
vessel_df['BaseDateTime'] = pd.to_datetime(vessel_df['BaseDateTime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
# Filter vessel data to only include rows from September 2023
september_vessels = vessel_df[(vessel_df['BaseDateTime'].dt.month == 9) & (vessel_df['BaseDateTime'].dt.year == 2022)]
subset_vessels = september_vessels[10000:12000]

# Strip any whitespace from the 'Status of System' column
hurricane_df['Status of System'] = hurricane_df['Status of System'].str.strip()
# Now apply the filter to get only hurricanes
# hurricane_only = hurricane_df[hurricane_df['Status of System'] == 'HU']


# Center the map off the U.S. East Coast
map_center = [31.0, -75.0]  # This latitude and longitude roughly centers on the East Coast
mymap = folium.Map(location=map_center, zoom_start=5)

# Iterate through hurricane dataset to plot hurricane locations
for idx, row in hurricane_df.iterrows():
    hurricane_location = [row['Latitude'], row['Longitude']]
    folium.Marker(
        location=hurricane_location,
        popup=(f"Hurricane: {row['Name']} | "
               f"Status: {row['Status of System']} | "
               f"Wind Speed: {row['Maximum Sustained Wind']} knots"),
        icon=folium.Icon(color='red', icon='cloud')
    ).add_to(mymap)

# Iterate through vessel dataset to plot vessel locations
for idx, row in subset_vessels.iterrows():
    vessel_location = [row['LAT'], row['LON']]
    folium.Marker(
        location=vessel_location,
        popup=(f"Vessel: {row['VesselName']} | "
               f"Type: {row['VesselType']} | "
               f"SOG: {row['SOG']} knots"),
        icon=folium.Icon(color='blue', icon='ship')
    ).add_to(mymap)

# Save map as an HTML file and display it
mymap.save("hurricane_vessel_map.html")

# Optionally open the map in your default web browser
import webbrowser
webbrowser.open("hurricane_vessel_map.html")




Python(68885) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


True

In [134]:
september_vessels.shape

(12061, 18)

In [135]:
hurricane_df["Status of System"].value_counts()

Status of System
EX    101
TS    100
HU     71
LO     55
TD     19
Name: count, dtype: int64

In [136]:
hurricane_only = hurricane_df[hurricane_df['Status of System'] == 'HU']
hurricane_only

,Unnamed: 0,Name,Num Entries,Year,Time,Record Identifier,Status of System,Latitude,Longitude,Maximum Sustained Wind,...,34-knot Wind Radii SW,34-knot Wind Radii NW,50-knot Wind Radii NE,50-knot Wind Radii SE,50-knot Wind Radii SW,50-knot Wind Radii NW,64-knot Wind Radii NE,64-knot Wind Radii SE,64-knot Wind Radii SW,64-knot Wind Radii NW
15,56209,FRANKLIN,86,9/1/2023,0,,HU,36.4,-59.7,75.0,...,140.0,80.0,80.0,80.0,70.0,40.0,40.0,50.0,40.0,20.0
16,56210,FRANKLIN,86,9/1/2023,600,,HU,37.4,-58.2,70.0,...,130.0,150.0,70.0,80.0,70.0,60.0,40.0,40.0,40.0,30.0
17,56211,FRANKLIN,86,9/1/2023,1200,,HU,38.2,-56.6,65.0,...,120.0,220.0,50.0,90.0,80.0,80.0,30.0,30.0,50.0,50.0
149,56402,LEE,55,9/6/2023,1800,,HU,14.6,-46.1,65.0,...,50.0,70.0,40.0,30.0,20.0,30.0,15.0,0.0,0.0,0.0
150,56403,LEE,55,9/7/2023,0,,HU,15.1,-47.1,70.0,...,50.0,70.0,40.0,30.0,20.0,30.0,15.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,56523,NIGEL,46,9/21/2023,0,,HU,38.3,-51.6,80.0,...,100.0,130.0,90.0,80.0,60.0,70.0,50.0,40.0,35.0,40.0
268,56524,NIGEL,46,9/21/2023,600,,HU,40.0,-49.2,75.0,...,110.0,130.0,80.0,80.0,60.0,70.0,50.0,40.0,30.0,40.0
269,56525,NIGEL,46,9/21/2023,1200,,HU,41.3,-46.3,75.0,...,110.0,130.0,80.0,80.0,60.0,70.0,50.0,40.0,30.0,40.0
270,56526,NIGEL,46,9/21/2023,1800,,HU,42.8,-42.8,70.0,...,180.0,110.0,100.0,110.0,80.0,60.0,60.0,60.0,40.0,30.0


In [137]:
import folium

# Define the latitude and longitude boundaries
min_lat = 25  # Southernmost point (Florida)
max_lat = 45  # Northernmost point (Maine)
min_lon = -80  # Western boundary (closer to the U.S. coast)
max_lon = -45  # Eastern boundary (further into the Atlantic Ocean)

# Center the map near the U.S. East Coast
map_center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]  # Center between these bounds
mymap = folium.Map(location=map_center, zoom_start=5)

# Add a rectangle representing the area defined by the coordinates
folium.Rectangle(
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    color='blue',
    fill=True,
    fill_opacity=0.2,
    popup='Longitude -80°W to -65°W, Latitude 25°N to 45°N'
).add_to(mymap)

# Save map as an HTML file
mymap.save("us_east_coast_area.html")

# Optionally open the map in your default web browser
import webbrowser
webbrowser.open("us_east_coast_area.html")


Python(68886) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


True